In [6]:
import pandas as pd
import datetime
import numpy as np
from nombres import normalizacion_nombres
from nacionalidades import normalizacion_paises
from institucion import normalizacion_universidades, region_institucion, macrozona_region

### Funciones utiles

In [7]:
ruta_base = "eca/datos-separados/"
ruta_final = "eca/finales/"
lista_archivos = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60']

In [8]:
##Para facilitar el procesamiento de nombres, se decidio eliminar todos los
##tildes y caracteres especiales (dierecis).
def limpiar_caracteres_especiales( string ):
    if 'á' in string or 'ä' in string:
        string = string.replace('ä', 'a').replace('á', 'a')
    if 'é' in string or 'ë' in string: 
        string = string.replace('é', 'e').replace('ë', 'e')
    if 'í' in string or 'ï' in string:
        string = string.replace('í', 'i').replace('ï', 'i')
    if 'ó' in string or 'ö' in string:
        string = string.replace('ó', 'o').replace('ö', 'o')
    if 'ú' in string or 'ü' in string:
        string = string.replace('ú', 'u').replace('ü', 'u')
    return string

##La idea es que dada la descripcion de la funcion, buscar palabras claves que permitan
##determinar que cargo tuvo dentro de la expedicion, al final, se aplica una jerarquizacion para solo
##dejar un cargo
def limpiar_funciones(x):
    if type(x) != str:
        return x
    
    final = ""

    if 'noinf' == x:
        return 'noinf'

    if 'ingeniero' in x.lower():
        if final!="": final+="-"
        final+="ING"
     
    if 'jefe' in x.lower() or 'jefa' in x.lower() or 'leader' in x.lower():
        if final!="": final+="-"
        final+="COO"
        
    elif 'coordi' in x.lower():
        if final!="": final+="-"
        final+="COO"

    if 'invest' in x.lower() or 'cientifico' in x.lower() or 'botanico' in x.lower() or 'microbiologa' in x.lower() or 'oceanografia fisica' in x.lower():
        if final!="": final+="-"
        final+="INV"

    if 'montaña' in x.lower() or 'mountain' in x.lower():
        if final!="": final+="-"
        final+="TECN"
        
    elif 'tecnico' in x.lower() or 'tecnico' in x.lower() or 'technician' in x.lower() or 'encargado' in x.lower() or 'mantencion' in x.lower():
        if final!="": final+="-"
        final+="TECN"

    elif 'buzo' in x.lower() or 'buceo' in x.lower():
        if final!="": final+="-"
        final+="TECN"

    if 'estudiante' in x.lower() or 'posdoc' in x.lower():
        if final!="": final+="-"
        final+="EST"
        
    elif 'tesis' in x.lower():
        if final!="": final+="-"
        final+="EST"

    if 'asisten' in x.lower() or 'ayudante' in x.lower() or 'apoyo' in x.lower():
        if final!="": final+="-"
        final+="AST"
        
    elif ('toma' in x.lower() or 'colec' in x.lower()):
        if final!="": final+="-"
        final+="AST"
        
    elif 'muestr' in x.lower() or 'procesamiento' in x.lower() or 'analisis' in x.lower() or 'sampling' in x.lower():
        if final!="": final+="-"
        final+="AST"

    if final=="": return "OTRO"
    
    if 'INV' in final: return 'INV'
    elif 'COO' in final: return 'COO'
    elif 'EST' in final: return 'EST'
    elif 'ING' in final: return 'ING'
    elif 'AST' in final: return 'AST'
    else: return 'TECN'

##La idea que dependiendo del codigo, se determine que instrumento se uso 
##para su financiamiento
def instrumento_financiamiento( codigo ):
    final = 'noinf'
    if codigo in ["MT", "MG", "DT", "DG", "RG", "AMP", "RT", "AM"]:
        final = "INACH"
    elif codigo in ["FP", "FI", "FR", "PR"]:
        final = "FONDECYT"
    elif codigo in ["FOVI", "FV"]:
        final = "FOMENTO"
    elif codigo in ["INT", "IN"]:
        final = "INTER"
    elif codigo in ["FE"]:
        final = "FONDEQUIP"
    elif codigo in ["NE"]:
        final = "NODO"
    elif codigo in ["AN"]:
        final = "ANILLO"
    elif codigo in ["MI", "IM"]:
        final = "MILENIO"
    elif codigo in ["FD", "FO"]:
        final = "FONDAP"
    elif codigo in ["CI", "PC", "AG", "PCI", "FIC"]:
        final = "CONICYT"
    elif codigo in ["CE"]:
        final = "CECs"
    elif codigo in ["CO"]:
        final = "CORFO"
    elif codigo in ["MA"]:
        final = "UMAG"
    elif codigo in ["OA"]:
        final = "OPERADORES ANTARTICOS"
    elif codigo in ["GU"]:
        final = "GLACIAR UNION"
    return final

##Aca se agrupan los instrumentos segun la institucion que las entrega
def institucion_instrumento( codigo ):
    sigla = codigo.split('-')[0]
    if sigla in ['GU']:
        return 'GLACIAR UNION'
    if sigla in ['OA']:
        return 'OP. ANTART'
    if sigla in ['CO']:
        return 'CORFO'
    if sigla in ['MA']:
        return 'UMAG'
    if sigla in ['CE']:
        return 'CECs'
    if sigla in ["AM", "DG", "DT", "MG", "MT", "RG", "RT", "AMP"]:
        return 'INACH'
    if sigla in ['AG', 'AN', 'CI', 'FD', 'FE', 'FI', 'FO', 'FOVI', 'FP', 'FR', 
        'FV', 'IM', 'IN', 'INT', 'NE', 'PC', 'PR', 'MI', "PCI", "FIC"]:
        return 'ANID'
    return 'noinf'

##La idea de esta funcion es detarminar si el codigo es de un proyecto
##de magister, doctorado, regular, internacional, externo
def descripcion_codigo_inach( codigo ):
    sigla = codigo.split('-')[0]
    if sigla in ['MT', 'MG']:
        return 'magister', 'tesis'
    elif sigla in ['DT', 'DG']:
        return 'doctorado', 'tesis'
    elif sigla in ['RG', 'RT']:
        return 'regular', 'noinf'
    elif sigla in ['INT', 'IN']:
        return 'internacional', 'noinf'
    else:
        return 'externo', 'noinf'

### Unificar datos todas las personas

In [ ]:
new_xls = []


for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list = pd.read_excel(f"{ruta_base}ECA {f}.xlsx").to_numpy().tolist()
    else:
        xls_list = pd.read_excel(f"{ruta_base}ECA {f}.xlsx", sheet_name=['personas'])['personas'].to_numpy().tolist()
    for row in xls_list:
        if pd.isna( row[0] ):
            pass
        ##Cosas que se buscar
        #codigo, titulo, pi, inst_pi, nombre+apellido, inst, nac, genero y funcion
        if f in ['50', '52']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero, funcion = row
        elif f in ['51']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero = row
            funcion = 'noinf'
        elif f in ['53']:
            codigo, pi, nombre, apellido, _, genero, inst_pi, titulo = row
            inst = 'noinf'
            nac = 'noinf'
            funcion = 'noinf'
        elif f in ['57']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ = row
            inst_pi = 'noinf'
            funcion = 'noinf'
        elif f in ['54']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ , inst_pi, _= row
            funcion = 'noinf'
        elif f in ['55', '56']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _ = row
            inst_pi = 'noinf'
        elif f in ['58', '59', '60']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _, inst_pi = row
        else:
            pass
        
        ##Limpieza de los datos
        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        proyecto, tesis = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        anio = 'noinf'
        if len( codigo.split('-') ) != 3:
            pass
        else:
            anio = f"20{ codigo.split('-')[2] }"

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        nombre = limpiar_caracteres_especiales( nombre.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        apellido = limpiar_caracteres_especiales( apellido.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        nombre = f"{ nombre } { apellido }".strip()

        inst = limpiar_caracteres_especiales( inst.lower().strip() )
        inst = normalizacion_universidades[ inst ]
        region_inst = region_institucion[inst]
        macro_inst = macrozona_region[region_inst]

        inst_pi = limpiar_caracteres_especiales( inst_pi.lower().strip() )
        inst_pi = normalizacion_universidades[ inst_pi ]
        region_inst_pi = region_institucion[inst_pi]
        macro_inst_pi = macrozona_region[region_inst_pi]

        nac = normalizacion_paises[ nac.strip().upper() ].lower().strip()

        funcion = limpiar_funciones( limpiar_caracteres_especiales( funcion.lower() ) )


        genero = genero.strip().upper()
        if genero == 'MASCULINO':
            genero = 'M'
        elif genero == 'FEMENINO':
            genero = 'F'
        elif genero == 'NOINF':
            genero = genero.lower()

        new_xls.append([f, anio, codigo, instrumento, proyecto, inst_instrumento, tesis, titulo, 
            pi, inst_pi, region_inst_pi, macro_inst_pi, 
            nombre, inst, region_inst, macro_inst, nac, genero, funcion])
    print(f"{f} done")

In [15]:
fecha = datetime.datetime.now()

full_dataframe = pd.DataFrame(np.array(new_xls), columns=['eca', 'anio', 'codigo', 'instrumento', 'inst-instrumento', 'tipo-proyecto-inach', 'tesis', 
    'titulo', 'pi', 'inst-pi', 'region-inst-pi', 'macro-inst-pi', 
    'nombre', 'inst', 'region-inst', 'macro-inst', 'nac', 'genero', 'funcion'])
full_dataframe.to_excel(f"{ruta_final}ECA todos-personas {fecha.day}.{fecha.month}.{fecha.year}.xlsx")

### Unificar datos todos los proyectos

In [ ]:
new_xls = []
codigos_ya_considerados = []

for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list = pd.read_excel(f"{ruta_base}ECA {f}.xlsx").to_numpy().tolist()
    else:
        xls_list = pd.read_excel(f"{ruta_base}ECA {f}.xlsx", sheet_name=['proyectos'])['proyectos'].to_numpy().tolist()
    for row in xls_list:
        if pd.isna( row[0] ):
            pass
        ##Cosas que se buscar
        #codigo, titulo, pi, inst_pi, ciudad
        if f in ['50', '52']:
            codigo, titulo, pi, inst_pi, _,_, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['51']:
            codigo, titulo, pi, inst_pi, _, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['53', '59', '60']:
            codigo, titulo, pi, inst_pi, _ = row
            ciudad = 'noinf'
        elif f in ['54', '58']:
            codigo, titulo, pi, inst_pi, ciudad, _ = row
        elif f in ['55', '56']:
            codigo, titulo, pi, _, _, _, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['57']:
            codigo, titulo, pi, _, _, _, _, _, _ = row
            inst_pi = 'noinf'
            ciudad = 'noinf'

        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        if codigo not in codigos_ya_considerados:
            proyecto, tesis = descripcion_codigo_inach( codigo )
            instrumento = instrumento_financiamiento( codigo.split('-')[0] )
            inst_instrumento = institucion_instrumento( codigo )

            titulo = limpiar_caracteres_especiales( titulo.lower() )

            pi = limpiar_caracteres_especiales( pi.lower() ).strip()
            if '/' in pi:
                pi = pi.split('/')[0].strip()
            elif '-' in pi:
                pi = pi.split('-')[0].strip()
            pi = normalizacion_nombres[pi]
            
            inst_pi = limpiar_caracteres_especiales( inst_pi.lower().strip() ).strip()
            inst_pi = normalizacion_universidades[ inst_pi ]
            region_inst_pi = region_institucion[inst_pi]
            macro_inst_pi = macrozona_region[region_inst_pi]

            
            new_xls.append([f, codigo, instrumento, proyecto, inst_instrumento, tesis, titulo, 
                pi, ciudad, inst_pi, region_inst_pi, macro_inst_pi, ])
            codigos_ya_considerados.append( codigo )
    print(f"{f} done")


In [20]:
fecha = datetime.datetime.now()

full_dataframe = pd.DataFrame(np.array(new_xls), columns=['eca', 'codigo', 'instrumento', 'inst-instrumento', 'tipo-proyecto-inach', 'tesis', 
    'titulo', 'pi', 'ciudad', 'inst-pi', 'region-inst-pi', 'macro-inst-pi'])
full_dataframe.to_excel(f"{ruta_final}ECA todos-proyectos {fecha.day}.{fecha.month}.{fecha.year}.xlsx")

### Unificar datos de todas las personas sin repeticion

### Unificar datos de todos los proyectos sin repeticion